# 평가 지표 정의 설명

본 프로젝트는 이진 분류(binary classification) 문제이며,
클래스 간 불균형(class imbalance)이 존재하므로 다음 지표를 사용한다.

1) Accuracy
   - 전체 문장 중 모델이 맞게 예측한 비율
   - Accuracy = (TP + TN) / (전체)

2) Precision (class-wise)
   - 모델이 "핵심문장(1)"이라고 예측한 것 중 실제로 1인 비율
   - Precision = TP / (TP + FP)

3) Recall (class-wise)
   - 실제 핵심문장(1) 중 모델이 1이라고 맞게 예측한 비율
   - Recall = TP / (TP + FN)

4) F1-score (class-wise)
   - Precision과 Recall의 조화 평균
   - F1 = 2 * (Precision * Recall) / (Precision + Recall)

5) Macro F1 (최종 평가 지표)
   - 클래스별 F1을 동일 가중치로 평균
   - 클래스 불균형이 존재할 때 더 합리적인 지표
   - 본 과제의 핵심 평가 지표로 사용

# 평가 프로토콜 설명

본 프로젝트의 최종 모델 평가는 다음 프로토콜을 따른다.

1) 데이터 분할
   - 전체 데이터(115,890 문장)를 weak_label 기준 Stratified Split
   - Train 80% (92,712)
   - Validation 10% (11,589)
   - Test 10% (11,589)

2) 학습
   - Train → 모델 학습
   - Validation → Hyperparameter와 모델 구조 선택
   - 최종 선택된 모델은 Validation 기준 성능이 가장 좋은 MLP Classifier

3) 최종 평가는 오직 Test set에서만 수행
   - Training/Validation 과정에서 사용되지 않은 데이터
   - 모델의 일반화 성능(Generalization)을 평가

4) 추가 평가
   - 사람이 직접 라벨링한 Human Label 200개에도 평가 수행
   - Weak label과 Human label의 차이를 분석하는 것이 핵심

위 과정을 통해 모델의 성능을 객관적이고 재현 가능하게 평가하였다.

# 데이터 로드 및 준비

In [2]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, f1_score, classification_report

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%cd /content/drive/MyDrive/ML_Project

Mounted at /content/drive
/content/drive/MyDrive/ML_Project


In [3]:
test = pd.read_csv("weak_test.csv")

X_test = test["clean_sentence"].fillna("")
y_test = test["weak_label"]

print("Test 데이터 크기:", test.shape)
test.head()

Test 데이터 크기: (11589, 5)


,lecture_id,major,sentence,clean_sentence,weak_label
0,C14600,comp,3 6 9 18 여섯 개 있겠죠.,3 6 9 18 여섯 개,0
1,C06299,comp,동전을 한번 던졌을 때 두 번째 동 던진 동전의 확률은 첫 번째 던진 동전하고 확률...,동전을 던졌을 때 두 번째 동 던진 동전의 확률은 첫 번째 던진 동전하고 확률하고 ...,0
2,C02199,comp,스텝 같은 경우도 얘는 스틸 마이너스 7 요런 값들이 이제 나오게 되네요?,스텝 같은 경우도 스틸 마이너스 7 값들이 나오게 되네요,0
3,C13849,comp,요까지 됐죠?,요까지 됐죠,0
4,C02510,comp,그렇게 정의를 했어요.,그렇게 정의를 했어요,0


In [4]:
model = joblib.load("final_model.pkl")
tfidf = joblib.load("tfidf_vectorizer.pkl")

print("모델, 벡터 로드 완료")

모델, 벡터 로드 완료


In [5]:
X_test_tfidf = tfidf.transform(X_test)
X_test_tfidf.shape

(11589, 20000)

# 1. Weak Test

In [6]:
test_pred = model.predict(X_test_tfidf)

acc_test = accuracy_score(y_test, test_pred)
f1_test = f1_score(y_test, test_pred, average="macro")

print("- Test 성능 평가 -")
print("Accuracy:", acc_test)
print("Macro F1:", f1_test)
print("\nClassification Report:")
print(classification_report(y_test, test_pred))

- Test 성능 평가 -
Accuracy: 0.9478816118733282
Macro F1: 0.6424938455723873

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.97      0.97     11222
           1       0.27      0.37      0.31       367

    accuracy                           0.95     11589
   macro avg       0.62      0.67      0.64     11589
weighted avg       0.96      0.95      0.95     11589



# 2. 직접(human) 라벨링 기반 평가

In [7]:
import pandas as pd

human = pd.read_csv("human_label.csv")

print("Human Label 데이터 크기:", human.shape)
human.head()

Human Label 데이터 크기: (200, 3)


,sent_id,sentence,label
0,11129,혹시 질문이 있으세요?,0
1,65730,그래서 어 리스트리드 (cascade)/(캐스케이드) 그다음에 기본 값으로 변경한 ...,1
2,65209,너무 (data)/(데이터)가 많으면 이 보여지는 화면이 좀 복잡하기 때문에 고객 ...,0
3,95416,글꼴은 굴림체 글자 크기는 11포인트 줄 간격은 18포인트 고정 어 내용 첫 줄은 ...,0
4,58033,그리고 그 문제에 피상적인 것을 해결하는 내용을 포함하여 근본적으로 이건 어떤 문제...,1


In [8]:
import re

def preprocess(text):
    text = str(text).lower()
    text = re.sub(r"[^가-힣a-zA-Z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# clean_sentence 새로
if "clean_sentence" not in human.columns:
    human["clean_sentence"] = human["sentence"].apply(preprocess)

X_human = human["clean_sentence"].fillna("")
y_human = human["label"]

In [9]:
X_human_tfidf = tfidf.transform(X_human)
X_human_tfidf.shape

(200, 20000)

In [11]:
human_pred = model.predict(X_human_tfidf)

acc_human = accuracy_score(y_human, human_pred)
f1_human = f1_score(y_human, human_pred, average="macro")

print("- Human Label Test 성능 -")
print("Accuracy:", acc_human)
print("Macro F1:", f1_human)

print("\nClassification Report:")
print(classification_report(y_human, human_pred))

- Human Label Test 성능 -
Accuracy: 0.68
Macro F1: 0.4903647077560121

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.97      0.80       133
           1       0.64      0.10      0.18        67

    accuracy                           0.68       200
   macro avg       0.66      0.54      0.49       200
weighted avg       0.67      0.68      0.59       200



# 최종 성능 비교

In [13]:
import pandas as pd

results = pd.DataFrame({
    "Dataset": ["Weak Test", "Human Test"],
    "Accuracy": [acc_test, acc_human],
    "Macro F1": [f1_test, f1_human]
})

results

,Dataset,Accuracy,Macro F1
0,Weak Test,0.947882,0.642494
1,Human Test,0.680000,0.490365
